In [1]:
from pipeline_functions import *

Using TensorFlow backend.
W1210 01:15:44.950538 140009864214336 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1210 01:15:45.015995 140009864214336 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1210 01:15:45.022873 140009864214336 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1210 01:15:45.432956 140009864214336 deprecation_wrapper.py:119] From /home/matteo/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compa

In [20]:
raw_data = "Michael Jordan was a basketball player on the Chicago Bulls."

In [21]:
ner_data = entity_recognition(raw_data)
cleaned_ner_data = text_cleaning(ner_data)

In [22]:
old_ner_date = ner_data
ner_data = cleaned_ner_data

In [53]:
# with open('../data/mod_anchor_candidates.pkl', 'rb') as f:
#     candidates = pickle.load(f)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


KeyboardInterrupt: 

In [23]:
ner_data

,entity,entity_start,entity_end,text,tokenized_vector
0,michael jordan,0,14,michael jordan was a basketball player on the ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,the chicago bulls,42,59,michael jordan was a basketball player on the ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]"


In [6]:
# candidate selection now

In [24]:
entity = ner_data.loc[0].entity
lsh_k = 10
anchor_k = 10

In [25]:
entity = candidate_preprocess(entity)

In [26]:
entity

'michael_jordan'

In [15]:
entity = candidate_preprocess(entity)

In [17]:
anchors_candidates = anchors_dict[entity][:anchor_k]

In [18]:
anchors_candidates

[(312.0, 0.901792),
 (89.0, 0.036336900000000005),
 (213710.0, 0.025043),
 (3854082.0, 0.0078125),
 (621231.0, 0.007611100000000001),
 (421253.0, 0.00712006),
 (1754545.0, 0.00589246),
 (493703.0, 0.00240385),
 (618355.0, 0.00180288),
 (368215.0, 0.00171863)]

In [19]:
anchors_candidates = [int(candidate[0]) for candidate in anchors_candidates]

In [20]:
anchors_candidates

[312, 89, 213710, 3854082, 621231, 421253, 1754545, 493703, 618355, 368215]

In [21]:
lsh_candidates = lsh_predict(entity, top_wikidata, 128, lsh_k, forest).tolist()

In [22]:
if anchors_candidates is None:
    anchors_candidates = []
if lsh_candidates is None:
    lsh_candidates = []

In [24]:
candidate_list = list(set(anchors_candidates + lsh_candidates))

skip above

In [27]:
ner_data['candidate_list'] = ner_data.apply(lambda i: get_candidate_list(i.entity, 10, 10), axis=1)

In [28]:
# my code starts here

In [29]:
ner_data

,entity,entity_start,entity_end,text,tokenized_vector,candidate_list
0,michael jordan,0,14,michael jordan was a basketball player on the ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[65029442, 6831718, 3308007, 6831721, 7412073,..."
1,the chicago bulls,42,59,michael jordan was a basketball player on the ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]","[19798658, 3223043, 128109, 2410223, 21528111,..."


In [30]:
# NEED TO ADD PREPROCESSING
# with open('../data/knowledge_graph_data/idx2id_entity.pickle', 'rb') as f:
#     idx2id = pickle.load(f)
    
# id2idx = {value: key for key, value in idx2id.items()}

# with open('../data/knowledge_graph_data/id2idx_entity.pickle', 'wb') as f:
#     pickle.dump(id2idx, f)

In [31]:
# add package import
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [32]:
# add to all import 
with open("../data/word2idx/word2idx.pkl", "rb") as f:
    #Map any unknown word to UNKNOWN, which has index 1
    word2idx = pickle.load(f)
    
with open('../data/knowledge_graph_data/id2idx_entity.pickle', 'rb') as f:
    id2idx_graph = pickle.load(f)
    
with open('../data/wiki_id_to_page.pkl', 'rb') as f:
    wiki_id_to_page = pickle.load(f)

graph_embedding = np.load('../data/knowledge_graph_data/wiki_DistMult_entity.npy') 
# add param
# window length on each side
WINDOW_LENGTH = 10
dict_size = len(word2idx)

In [33]:
NED_model = load_model("final_NED_model.h5")

In [34]:
window_length = WINDOW_LENGTH

In [35]:
ner_data['predicted_wikidata'] = 29485

In [38]:
for key, value in ner_data.iterrows():
    entity = value.entity
    split_text = value.text.split()
    candidate_list = value.candidate_list
    
    assert len(value.tokenized_vector) == len(split_text)
    
    # find context
    entity_idx = np.where(value.tokenized_vector)[0][0]
    context = split_text[max(0, entity_idx - window_length) : entity_idx] +\
                  split_text[entity_idx + 1 : entity_idx + window_length + 1]
    context_word2idx = [word2idx.get(word, 1) for word in context]
    
    # inputs
    lstm_input = pad_sequences(np.array([context_word2idx]), maxlen = window_length * 2, padding = 'post')
    kge_candidates = [
        graph_embedding[id2idx_graph[entity_id]]
        if entity_id in id2idx_graph else None
        for entity_id in candidate_list
    ]
    
    # prediction
    candidate_probs = [
        NED_model.predict([lstm_input, kge_candidate.reshape(1,-1)])[0][0]
        if not kge_candidate is None else 0.0
        for kge_candidate in kge_candidates
    ]
    

    ner_data.loc[key,'predicted_wikidata'] = candidate_list[np.argmax(candidate_probs)]
    dict_df_info = {}
    dict_df_info['Candidate'] = [wiki_id_to_page[candidate_list[idx]] for idx in np.argsort(candidate_probs)[::-1]]
    dict_df_info['Probability'] = [candidate_probs[idx] for idx in np.argsort(candidate_probs)[::-1]]
    display(pd.DataFrame(dict_df_info))
    print("Candidate List: {}".format("; ".join(dict_df_info['Candidate'])))
    print(len(dict_df_info['Candidate']))
    print(wiki_id_to_page[ner_data.loc[key,'predicted_wikidata']])

,Candidate,Probability
0,Michael_Jordan,0.697914
1,Michael_B._Jordan,0.681982
2,Michael_H._Jordan,0.079562
3,Samuel_Jordan,0.035696
4,Michael_Jordan's_Steakhouse,0.031856
5,Michael_Jordan_statue,0.018416
6,Michael_I._Jordan,0.013805
7,Samuel_M._Jordan,0.009411
8,Michael_Jordan_(offensive_lineman),0.008144
9,Michael_John,0.004882


Candidate List: Michael_Jordan; Michael_B._Jordan; Michael_H._Jordan; Samuel_Jordan; Michael_Jordan's_Steakhouse; Michael_Jordan_statue; Michael_I._Jordan; Samuel_M._Jordan; Michael_Jordan_(offensive_lineman); Michael_John
10
Michael_Jordan


,Candidate,Probability
0,Chicago_Bulls,0.938144
1,The_Pinnacle_(Chicago),0.144753
2,The_Chicago_Maroon,0.076415
3,The_Chicagoan,0.030247
4,The_Earl_of_Chicago,0.027759
5,The_Chicago_Kid,0.015988
6,Holy_Name_Cathedral_(Chicago),0.008239
7,The_Innocents_of_Chicago,0.006233


Candidate List: Chicago_Bulls; The_Pinnacle_(Chicago); The_Chicago_Maroon; The_Chicagoan; The_Earl_of_Chicago; The_Chicago_Kid; Holy_Name_Cathedral_(Chicago); The_Innocents_of_Chicago
8
Chicago_Bulls


In [37]:
ner_data

,entity,entity_start,entity_end,text,tokenized_vector,candidate_list,predicted_wikidata
0,michael jordan,0,14,michael jordan was a basketball player on the ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[65029442, 6831718, 3308007, 6831721, 7412073,...",41421
1,the chicago bulls,42,59,michael jordan was a basketball player on the ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]","[19798658, 3223043, 128109, 2410223, 21528111,...",128109


['IOS',
 'IPhone',
 'IPhone_5',
 'IPhone_(1st_generation)',
 'IPhone_6',
 'IPhone_3G',
 'Triphone']

In [67]:
np.argmax(candidate_probs)

2

'Apple_Inc.'